In [ ]:
import torch
print(torch.__version__)

In [ ]:
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import matplotlib.pyplot as plt
from time import time
import numpy as np
import pandas as pd
from six.moves import urllib
import random
from skimage.util import random_noise
from math import log10
import torch.nn.functional as F
import math

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
!nvidia-smi

In [ ]:
# Read the train and test sets of CIFAR-10 data
default_transform = transforms.Compose([transforms.ToTensor()])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=default_transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=default_transform)
print("Training set size:", len(cifar_trainset))
print("Test set size:", len(cifar_testset))

In [ ]:
# Initialize data loader functions
BATCH_SIZE = 64
train_dataLoader = DataLoader(cifar_trainset, batch_size=BATCH_SIZE, shuffle=True)
test_dataLoader = DataLoader(cifar_testset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Validate shape of the input images
dataiter = iter(train_dataLoader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

In [ ]:
def conv1x1(in_channels, out_channels, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

In [ ]:
def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, groups=groups, bias=False)

## Variant 2 (Conv2D --> ReLU --> IC)

In [ ]:
def IC(inputs, p=0.03):
  y = nn.Sequential(
      nn.BatchNorm2d(inputs),
      nn.Dropout(p))

  # y = nn.Dropout(p)(y)

  return y

In [ ]:
class Bottleneck_IC2(nn.Module):
    expansion = 4  # # output cahnnels / # input channels

    def __init__(self, inplanes, outplanes, stride=1):
        assert outplanes % self.expansion == 0
        super(Bottleneck_IC2, self).__init__()
        self.inplanes = inplanes
        self.outplanes = outplanes
        self.bottleneck_planes = int(outplanes / self.expansion)
        self.stride = stride

        self._make_layer()

    def _make_layer(self):
        # conv 1x1
        # self.bn1 = nn.BatchNorm2d(self.inplanes)
        
        self.conv1 = nn.Conv2d(self.inplanes, self.bottleneck_planes,
                               kernel_size=1, stride=self.stride, bias=False)
        self.IC1 = IC(self.bottleneck_planes)
        # conv 3x3
        # self.bn2 = nn.BatchNorm2d(self.bottleneck_planes)
        
        self.conv2 = nn.Conv2d(self.bottleneck_planes, self.bottleneck_planes,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.IC2 = IC(self.bottleneck_planes)
        # conv 1x1
        # self.bn3 = nn.BatchNorm2d(self.bottleneck_planes)
        
        self.conv3 = nn.Conv2d(self.bottleneck_planes, self.outplanes, kernel_size=1,
                               stride=1)
        self.IC3 = IC(self.outplanes)

        if self.inplanes != self.outplanes:
            self.shortcut = nn.Conv2d(self.inplanes, self.outplanes, kernel_size=1,
                                      stride=self.stride, bias=False)
        else:
            self.shortcut = None
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
  
        out = self.conv1(x)
        out = self.relu(out)
        out = self.IC1(out)

        out = self.conv2(out)
        out = self.relu(out)
        out = self.IC2(out)

        out = self.conv3(out)
        out = self.relu(out)
        out = self.IC3(out)

        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, depth, output_classes=10):
        assert (depth - 2) % 9 == 0  # 164 or 1001
        super(ResNet_alt, self).__init__()
        n = int((depth - 2) / 9)
        nstages = [16, 64, 128, 256]
        # one conv at the beginning (spatial size: 32x32)
        self.conv1 = nn.Conv2d(3, nstages[0], kernel_size=3, stride=1,
                               padding=1, bias=False)

        # use `block` as unit to construct res-net
        # Stage 0 (spatial size: 32x32)
        self.layer1 = self._make_layer(block, nstages[0], nstages[1], n)
        # Stage 1 (spatial size: 32x32)
        self.layer2 = self._make_layer(block, nstages[1], nstages[2], n, stride=2)
        # Stage 2 (spatial size: 16x16)
        self.layer3 = self._make_layer(block, nstages[2], nstages[3], n, stride=2)
        # Stage 3 (spatial size: 8x8)
        self.bn = nn.BatchNorm2d(nstages[3])
        self.relu = nn.ReLU(inplace=True)
        # classifier
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(nstages[3], output_classes)

        # weight initialization
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, inplanes, outplanes, nstage, stride=1):
        layers = []
        layers.append(block(inplanes, outplanes, stride))
        for i in range(1, nstage):
            layers.append(block(outplanes, outplanes, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.relu(self.bn(x))

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
def get_accuracy(model, dataloader):
  """
    Function to compute accuracy given a model (of class nn) and a dataloader object
  """
  
  model.eval()
  correct_predictions = 0
  with torch.no_grad():
    for images, labels in dataloader:
      imgs = images.to(device)
      lbls = labels.to(device)
      # images = images.view(images.shape[0], -1)
      output = model(imgs)
      _, predicted = torch.max(output.data, 1)
      correct_predictions += (predicted == lbls).sum().item()
  accuracy = (correct_predictions / len(dataloader.dataset)) * 100
  return(accuracy)


In [ ]:
def train_network(model, num_epochs, learning_rate, train_dataLoader, test_dataLoader, lr_update_rule):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  train_accuracy_list, test_accuracy_list, train_loss_list = [], [], [] 
  for epoch in range(num_epochs):
    running_loss = 0
    for images, labels in train_dataLoader:
      imgs = images.to(device)
      lbls = labels.to(device)
  
      # Training step
      optimizer.zero_grad()
      out = model(imgs)
      loss = criterion(out, lbls)
      
      # Backpropagate loss
      loss.backward()
      
      # Optimize weights
      optimizer.step()
      
      running_loss += loss.item()
    
    train_loss = running_loss/len(train_dataLoader)
    train_loss_list.append(train_loss)
    train_accuracy = get_accuracy(model, train_dataLoader)
    train_accuracy_list.append(train_accuracy)
    
    test_accuracy = get_accuracy(model, test_dataLoader)
    test_accuracy_list.append(test_accuracy)
    print("Epoch: {} \t Training loss: {} \t Training accuracy: {} \t Test accuracy: {}".format(epoch, train_loss, train_accuracy, test_accuracy))
    
  return model, train_accuracy_list, test_accuracy_list, train_loss_list

In [ ]:
# Hyper-parameters
EPOCHS = 100
lr = 0.001
lr_update = {80:0.0001, 120:0.00001, 160:0.000001}

In [ ]:
resnet110B_ic2 = ResNet(Bottleneck_IC2, 110,10).to(device)
resnet110B_ic2, train_acc_110B_ic2, test_acc_110B_ic2, train_loss_110B_ic2 = train_network(model=resnet110B_ic2,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

In [ ]:
import pandas as pd
metric_data_ic2 = pd.DataFrame({'Epoch': range(1,EPOCHS+1), 'Train_Acc': train_acc_110B_ic2, 'Test_Acc': test_acc_110B_ic2, 'Train_Loss': train_loss_110B_ic2})
metric_data_ic2.to_csv('ResNet110_B_ic2.csv', index=False)
torch.save(resnet110B_ic2, 'ResNet110_B_ic2.pt')